In [1]:
from keras.layers import *
from keras.datasets import mnist
from keras.models import Model
from keras.optimizers import Adam
from keras.datasets import mnist
import numpy as np
import imageio
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

Using TensorFlow backend.


In [10]:
class A:
    def __init__(self):
        print('a')

class B:
    def __init__(self):
        print('b')
        
class C(A, B):
    def __init__(self):
        super(C, self).__init__()

c = C()

a


In [8]:
from keras.preprocessing import sequence
from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.datasets import imdb


class VIB(Model):
    """变分信息瓶颈层
    """
    def __init__(self, lamb, **kwargs):
        self.lamb = lamb
        super(VIB, self).__init__(**kwargs)
        self.ds_mean = Dense(128)
        self.ds_log_var = Dense(128)
    def call(self, inputs):
        seq = inputs
        z_mean = self.ds_mean(seq)
        z_log_var = self.ds_log_var(seq)
        u = K.random_normal(shape=K.shape(z_mean))
        kl_loss = - 0.5 * K.sum(K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), 0))
        self.add_loss(self.lamb * kl_loss)
        u = K.in_train_phase(u, 0.)
        return z_mean + K.exp(z_log_var / 2) * u
    def compute_output_shape(self, input_shape):
        return input_shape[0]


max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
epochs = 5


(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

class VIB(Layer):
    """变分信息瓶颈层
    """
    def __init__(self, lamb, **kwargs):
        self.lamb = lamb
        super(VIB, self).__init__(**kwargs)
    def call(self, inputs):
        z_mean, z_log_var = inputs
        u = K.random_normal(shape=K.shape(z_mean))
        kl_loss = - 0.5 * K.sum(K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), 0))
        self.add_loss(self.lamb * kl_loss)
        u = K.in_train_phase(u, 0.)
        return z_mean + K.exp(z_log_var / 2) * u
    def compute_output_shape(self, input_shape):
        return input_shape[0]


max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
epochs = 5


(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


seq_in = Input(shape=(maxlen,))
seq = seq_in

seq = Embedding(max_features, embedding_dims)(seq)
seq = Conv1D(filters,
             kernel_size,
             padding='valid',
             activation='relu',
             strides=1)(seq)
seq = GlobalMaxPooling1D()(seq)

z_mean = Dense(128)(seq)
z_log_var = Dense(128)(seq)
seq = VIB(0.1)([z_mean, z_log_var])
seq = Dense(1, activation='sigmoid')(seq)

model = Model(seq_in, seq)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
validation_data=(x_test, y_test))
"""
# 去掉VIB后的模型
seq_in = Input(shape=(maxlen,))
seq = seq_in
seq = Embedding(max_features, embedding_dims)(seq)
seq = Conv1D(filters,
             kernel_size,
             padding='valid',
             activation='relu',
             strides=1)(seq)
seq = GlobalMaxPooling1D()(seq)
seq = Dense(128, activation='relu')(seq)
seq = Dense(1, activation='sigmoid')(seq)
model = Model(seq_in, seq)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
"""

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 7s 262us/step - loss: 0.3911 - acc: 0.8304 - val_loss: 0.2895 - val_acc: 0.8949
Epoch 2/5
25000/25000 [==============================] - 6s 244us/step - loss: 0.2187 - acc: 0.9291 - val_loss: 0.3000 - val_acc: 0.8871
Epoch 3/5
25000/25000 [==============================] - 6s 245us/step - loss: 0.1193 - acc: 0.9714 - val_loss: 0.3869 - val_acc: 0.8668
Epoch 4/5
25000/25000 [==============================] - 6s 244us/step - loss: 0.0695 - acc: 0.9901 - val_loss: 0.3851 - val_acc: 0.8829
Epoch 5/5
25000/25000 [==============================] - 6s 244us/step - loss: 0.0537 - acc: 0.9934 - val_loss: 0.6312 - val_acc: 0.8479


"\n# 去掉VIB后的模型\nseq_in = Input(shape=(maxlen,))\nseq = seq_in\nseq = Embedding(max_features, embedding_dims)(seq)\nseq = Conv1D(filters,\n             kernel_size,\n             padding='valid',\n             activation='relu',\n             strides=1)(seq)\nseq = GlobalMaxPooling1D()(seq)\nseq = Dense(128, activation='relu')(seq)\nseq = Dense(1, activation='sigmoid')(seq)\nmodel = Model(seq_in, seq)\nmodel.compile(loss='binary_crossentropy',\n              optimizer='adam',\n              metrics=['accuracy'])\nmodel.fit(x_train, y_train,\n          batch_size=batch_size,\n          epochs=epochs,\n          validation_data=(x_test, y_test))\n"

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
label0idx = [idx for idx in range(y_train.shape[0]) if y_train[idx]==0]
label1idx = [idx for idx in range(y_train.shape[0]) if y_train[idx]==1]
print(len(label0idx))
print(len(label1idx))
x_0 = x_train[label0idx]/255
x_0 = x_0.reshape(x_0.shape[0], -1)
x_0 = np.random.randn(*x_0.shape)
x_1 = x_train[label1idx]/255
x_1 = x_1.reshape(x_1.shape[0], -1)
print(x_0.shape)
print(x_1.shape)

img_dim = 784
img_sz = 28

(60000, 28, 28)
(60000,)
5923
6742
(5923, 784)
(6742, 784)


In [9]:
class Shuffle(Layer):
    """打乱层，提供两种方式打乱输入维度
    一种是直接反转，一种是随机打乱，默认是直接反转维度
    """
    def __init__(self, idxs=None, mode='reverse', **kwargs):
        super(Shuffle, self).__init__(**kwargs)
        self.idxs = idxs
        self.mode = mode
    def call(self, inputs):
        v_dim = K.int_shape(inputs)[-1]
        if self.idxs == None:
            self.idxs = list(range(v_dim))
            if self.mode == 'reverse':
                self.idxs = self.idxs[::-1]
            elif self.mode == 'random':
                np.random.shuffle(self.idxs)
        inputs = K.transpose(inputs)
        outputs = K.gather(inputs, self.idxs)
        outputs = K.transpose(outputs)
        return outputs
    def inverse(self):
        v_dim = len(self.idxs)
        _ = sorted(zip(range(v_dim), self.idxs), key=lambda s: s[1])
        reverse_idxs = [i[0] for i in _]
        return Shuffle(reverse_idxs)


class SplitVector(Layer):
    """将输入分区为两部分，交错分区
    """
    def __init__(self, **kwargs):
        super(SplitVector, self).__init__(**kwargs)
    def call(self, inputs):
        v_dim = K.int_shape(inputs)[-1]
        inputs = K.reshape(inputs, (-1, v_dim//2, 2))
        return [inputs[:,:,0], inputs[:,:,1]]
    def compute_output_shape(self, input_shape):
        v_dim = input_shape[-1]
        return [(None, v_dim//2), (None, v_dim//2)]
    def inverse(self):
        layer = ConcatVector()
        return layer


class ConcatVector(Layer):
    """将分区的两部分重新合并
    """
    def __init__(self, **kwargs):
        super(ConcatVector, self).__init__(**kwargs)
    def call(self, inputs):
        inputs = [K.expand_dims(i, 2) for i in inputs]
        inputs = K.concatenate(inputs, 2)
        return K.reshape(inputs, (-1, np.prod(K.int_shape(inputs)[1:])))
    def compute_output_shape(self, input_shape):
        return (None, sum([i[-1] for i in input_shape]))
    def inverse(self):
        layer = SplitVector()
        return layer


class AddCouple(Layer):
    """加性耦合层
    """
    def __init__(self, isinverse=False, **kwargs):
        self.isinverse = isinverse
        super(AddCouple, self).__init__(**kwargs)
    def call(self, inputs):
        part1, part2, mpart1 = inputs
        if self.isinverse:
            return [part1, part2 + mpart1] # 逆为加
        else:
            return [part1, part2 - mpart1] # 正为减
    def compute_output_shape(self, input_shape):
        return [input_shape[0], input_shape[1]]
    def inverse(self):
        layer = AddCouple(True)
        return layer


class Scale(Layer):
    """尺度变换层
    """
    def __init__(self, **kwargs):
        super(Scale, self).__init__(**kwargs)
    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(1, input_shape[1]),
                                      initializer='glorot_normal',
                                      trainable=True)
        super(Scale, self).build(input_shape)
    def call(self, inputs):
        self.add_loss(-K.sum(self.kernel)) # 对数行列式
        return K.exp(self.kernel) * inputs
    def inverse(self):
        scale = K.exp(-self.kernel)
        return Lambda(lambda x: scale * x)


def build_basic_model(v_dim):
    """基础模型，即加性耦合层中的m
    """
    _in = Input(shape=(v_dim,))
    _ = _in
    for i in range(5):
        _ = Dense(1000, activation='relu')(_)
    _ = Dense(v_dim, activation='relu')(_)
    return Model(_in, _)

def build_nice_model():
    shuffle1 = Shuffle()
    shuffle2 = Shuffle()
    shuffle3 = Shuffle()
    shuffle4 = Shuffle()

    split = SplitVector()
    couple = AddCouple()
    concat = ConcatVector()
    scale = Scale()

    basic_model_1 = build_basic_model(img_dim//2)
    basic_model_2 = build_basic_model(img_dim//2)
    basic_model_3 = build_basic_model(img_dim//2)
    basic_model_4 = build_basic_model(img_dim//2)


    x_in = Input(shape=(img_dim,))
    x = x_in

    x = shuffle1(x)
    x1,x2 = split(x)
    mx1 = basic_model_1(x1)
    x1, x2 = couple([x1, x2, mx1])
    x = concat([x1, x2])

    x = shuffle2(x)
    x1,x2 = split(x)
    mx1 = basic_model_2(x1)
    x1, x2 = couple([x1, x2, mx1])
    x = concat([x1, x2])

    x = shuffle3(x)
    x1,x2 = split(x)
    mx1 = basic_model_3(x1)
    x1, x2 = couple([x1, x2, mx1])
    x = concat([x1, x2])

    x = shuffle4(x)
    x1,x2 = split(x)
    mx1 = basic_model_4(x1)
    x1, x2 = couple([x1, x2, mx1])
    x = concat([x1, x2])

    x = Activation('sigmoid')(x)

    model = Model(x_in, x)
    return model

In [10]:
#build g_model
g_model = build_nice_model()

#build d_model
x_in = Input(shape=(img_dim,))
x = x_in
x = Dense(img_dim, activation='relu')(x)
x = Dense(img_dim, activation='relu')(x)
x = Dense(1, use_bias=False)(x)
d_model = Model(x_in, x)

#build d_train_model

d_model.trainable = True
g_model.trainable = False
x_real_in = Input(shape=(img_dim,))
x_fake_in = Input(shape=(img_dim,))
x_real = x_real_in
x_fake = g_model(x_fake_in)
x_real_score = d_model(x_real)
x_fake_score = d_model(x_fake)
d_train_model = Model([x_real_in, x_fake_in],
                      [x_real_score, x_fake_score])
d_loss = x_real_score - x_fake_score
d_norm = 10 * K.mean(K.abs(x_real - x_fake), axis=[1])
d_loss = K.mean(- d_loss + 0.5 * d_loss**2 / d_norm)
d_train_model.add_loss(d_loss)
d_train_model.compile(optimizer=Adam(2e-4, 0.5), loss=None)
#build g_train_model
d_model.trainable = False
g_model.trainable = True
x_real_in = Input(shape=(img_dim,))
x_fake_in = Input(shape=(img_dim,))
x_real = x_real_in
x_fake = g_model(x_fake_in)
x_real_score = d_model(x_real)
x_fake_score = d_model(x_fake)
g_train_model = Model([x_real_in, x_fake_in],
                      [x_real_score, x_fake_score])
g_loss = K.mean(x_real_score - x_fake_score)
g_train_model.add_loss(g_loss)
g_train_model.compile(optimizer=Adam(2e-4, 0.5), loss=None)

#train
def gen_batch(x, batch_size):
    idx = np.random.choice(len(x), batch_size)
    return x[idx]

def sample(fpath):
    num = 9
    x_fake = gen_batch(x_0, num*num)
    x_fake_gen = g_model.predict(x_fake)
    x_fake = x_fake.reshape(num*num, img_sz, img_sz)
    x_fake_gen = x_fake_gen.reshape(num*num, img_sz, img_sz)
    x_real = gen_batch(x_1, num*num)
    x_real = x_real.reshape(num*num, img_sz, img_sz)
    output = np.zeros((3*num*img_sz, num*img_sz))
    st1 = num*img_sz
    st2 = 2*num*img_sz
    for i in range(num):
        for j in range(num):
            output[i*img_sz:(i+1)*img_sz, j*img_sz:(j+1)*img_sz] = x_real[i*num+j]
            output[st1+i*img_sz:st1+(i+1)*img_sz, j*img_sz:(j+1)*img_sz] = x_fake[i*num+j]
            output[st2+i*img_sz:st2+(i+1)*img_sz, j*img_sz:(j+1)*img_sz] = x_fake_gen[i*num+j]
    imageio.imwrite(fpath, output)

def build_classify_model():
    x_in = Input(shape=(img_dim,))
    x = x_in
    x = Dense(128, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(x_in, x)
    model.compile(optimizer=Adam(2e-4, 0.5), loss='binary_crossentropy', metrics='acc')
    return model

def classify():
    pass
    #model = build_classify_model()
    #x_train = 
    
def train(num_iters, batch_size, train_d_iter, sample_iter, log_iter, classify_iter):
    #0 -> 1
    for i in range(num_iters):
        x_fake = gen_batch(x_0, batch_size)
        x_real = gen_batch(x_1, batch_size)
        for j in range(train_d_iter):
            d_loss = d_train_model.train_on_batch([x_real, x_fake], None)
        g_loss = g_train_model.train_on_batch([x_real, x_fake], None)
        if i % log_iter == 0:
            print(f'g_loss = {g_loss} d_loss = {d_loss}')
        if i % sample_iter == 0:
            sample(f'sample_gan_mnist/iter_{i}.png')
        if i % classify_iter == 0:
            classify()
g_model.summary()
train(num_iters=5000,
      batch_size=64,
      train_d_iter=2,
      sample_iter=100,
      log_iter=10,
      classify_iter=500)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           (None, 784)          0                                            
__________________________________________________________________________________________________
shuffle_9 (Shuffle)             (None, 784)          0           input_25[0][0]                   
__________________________________________________________________________________________________
split_vector_3 (SplitVector)    [(None, 392), (None, 0           shuffle_9[0][0]                  
                                                                 shuffle_10[0][0]                 
                                                                 shuffle_11[0][0]                 
                                                                 shuffle_12[0][0]                 
__________

/share/nanlh/work/pprli/pprli_env/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/share/nanlh/work/pprli/pprli_env/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


g_loss = 2.9287095069885254 d_loss = -1.3750810623168945


g_loss = 1.5264897346496582 d_loss = -0.7147051692008972
g_loss = 1.3957486152648926 d_loss = -0.6951966881752014
g_loss = 1.3291044235229492 d_loss = -0.6926892399787903
g_loss = 1.3832979202270508 d_loss = -0.6755393743515015
g_loss = 1.3556251525878906 d_loss = -0.6820212602615356
g_loss = 1.332237720489502 d_loss = -0.6852498054504395
g_loss = 1.3584277629852295 d_loss = -0.6849483251571655
g_loss = 1.3455818891525269 d_loss = -0.6884958148002625
g_loss = 1.3637800216674805 d_loss = -0.6788007020950317


g_loss = 1.3372381925582886 d_loss = -0.6793755292892456
g_loss = 1.2349694967269897 d_loss = -0.6761980056762695
g_loss = 1.3727487325668335 d_loss = -0.6762375831604004
g_loss = 1.3082460165023804 d_loss = -0.6974611878395081
g_loss = 1.3749237060546875 d_loss = -0.6819202899932861
g_loss = 1.272760272026062 d_loss = -0.6750117540359497
g_loss = 1.2214986085891724 d_loss = -0.6884100437164307
g_loss = 1.3313957452774048 d_loss = -0.6765607595443726
g_loss = 1.3495017290115356 d_loss = -0.6759544610977173
g_loss = 1.331747055053711 d_loss = -0.6789999008178711


...

g_loss = 1.200497031211853 d_loss = -0.6137809753417969
g_loss = 1.2671797275543213 d_loss = -0.6269727349281311
g_loss = 1.220973253250122 d_loss = -0.616145133972168
g_loss = 1.2335243225097656 d_loss = -0.6225377321243286
g_loss = 1.2824804782867432 d_loss = -0.6406416893005371
g_loss = 1.1679399013519287 d_loss = -0.6087818145751953
g_loss = 1.2776196002960205 d_loss = -0.6368093490600586
g_loss = 1.2977344989776611 d_loss = -0.6342454552650452
g_loss = 1.2692663669586182 d_loss = -0.6251206398010254
g_loss = 1.234188437461853 d_loss = -0.6231746673583984


g_loss = 1.2777099609375 d_loss = -0.6338064670562744
g_loss = 1.2843295335769653 d_loss = -0.629057765007019
g_loss = 1.242414951324463 d_loss = -0.6232219934463501
g_loss = 1.2774535417556763 d_loss = -0.6285609602928162
g_loss = 1.2482413053512573 d_loss = -0.6257616281509399
g_loss = 1.2732579708099365 d_loss = -0.6385220885276794
g_loss = 1.25846529006958 d_loss = -0.6261402368545532
g_loss = 1.2612642049789429 d_loss = -0.6294412612915039
g_loss = 1.2786986827850342 d_loss = -0.6335234642028809
g_loss = 1.26106858253479 d_loss = -0.6307772397994995


g_loss = 1.2825124263763428 d_loss = -0.6401498317718506
g_loss = 1.2198600769042969 d_loss = -0.6186304092407227
g_loss = 1.2720344066619873 d_loss = -0.6237193942070007
g_loss = 1.2603130340576172 d_loss = -0.628502607345581
g_loss = 1.3017868995666504 d_loss = -0.641263484954834
g_loss = 1.2817717790603638 d_loss = -0.6298856139183044
g_loss = 1.243302822113037 d_loss = -0.6313893795013428
g_loss = 1.2446725368499756 d_loss = -0.6256345510482788
g_loss = 1.2214869260787964 d_loss = -0.6184892654418945
g_loss = 1.2872347831726074 d_loss = -0.6359715461730957


g_loss = 1.2853784561157227 d_loss = -0.6326954364776611
g_loss = 1.225445032119751 d_loss = -0.6166419982910156
g_loss = 1.2354698181152344 d_loss = -0.6245889067649841
g_loss = 1.2697432041168213 d_loss = -0.6338899731636047
g_loss = 1.2404873371124268 d_loss = -0.6265147924423218
g_loss = 1.2036064863204956 d_loss = -0.6090502738952637
g_loss = 1.221085786819458 d_loss = -0.6160999536514282
g_loss = 1.228010892868042 d_loss = -0.6238529086112976
g_loss = 1.2645370960235596 d_loss = -0.6328611969947815
g_loss = 1.2809667587280273 d_loss = -0.6410366892814636


g_loss = 1.2263171672821045 d_loss = -0.6180318593978882
g_loss = 1.23653244972229 d_loss = -0.6216540932655334
g_loss = 1.2016170024871826 d_loss = -0.6097781658172607
g_loss = 1.2560486793518066 d_loss = -0.6268234252929688
g_loss = 1.227988362312317 d_loss = -0.6170956492424011
g_loss = 1.266645908355713 d_loss = -0.629179060459137
g_loss = 1.257666826248169 d_loss = -0.6290581226348877
g_loss = 1.2352802753448486 d_loss = -0.6207190752029419
g_loss = 1.3566126823425293 d_loss = -0.6481856107711792
g_loss = 1.2631568908691406 d_loss = -0.630932092666626


In [ ]:
print(len(np.zeros((2, 3, 4))))